In [1]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [2]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/cells")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [7]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Cell
                OR n:CellType 
                OR n:CellPhenotype 
                OR n:CellPhenotypeCategory 
                OR n:CellularRegion
                OR n:NeuralStructure
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Cell) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:CellType) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:CellPhenotype) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:CellPhenotypeCategory) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:NeuralStructure) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Cell(name)
            DROP INDEX ON :CellType(name)
            DROP INDEX ON :CellPhenotype(name)
            DROP INDEX ON :CellPhenotypeCategory(name)
            DROP INDEX ON :NeuralStructure(name)
        ''')
        print("wipeddatabase")

wipeddatabase


In [8]:
# Adding Cell to graph from cell_types.csv

csv_file_path = "file:///%s" % get_csv_path("cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Cell { id: row.ID, name: row.Cell_type_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX ON :Cell(name)")
    session.run("CREATE CONSTRAINT ON (n:Cell) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Cell", session)

Added 107 nodes of type Cell


In [9]:
# Adding CellTypes to graph
cell_types = [("Neuron", "SAO:1417703748"), 
              ("Oligodendrocyte","SAO:844118448	"), 
              ("Microglia","SAO:789292116"), 
              ("Astrocyte","SAO:1394521419")
             ]
with driver.session() as session:
    for cell_name, cell_ont in cell_types:
        q = "CREATE (:CellType {{ name: '{name}', ontology: '{ont}' }})".format(name=cell_name, ont=cell_ont)
        session.run(q )
        
    get_count_of_type("CellType", session)

Added 4 nodes of type CellType


In [10]:
# Adding CellPhenotype from cell_phenotypes.csv

csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotype { id: row.ID, name: row.Phenotype, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellPhenotype) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellPhenotype(name)")
    session.run(query)
    get_count_of_type("CellPhenotype", session)

Added 107 nodes of type CellPhenotype


In [11]:
# Adding CellPhenotypeCategory from cell_phenotype_categories.csv

csv_file_path = "file:///%s" % get_csv_path("cell_phenotype_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotypeCategory { id:row.ID, name: row.Phenotype_category})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellPhenotype) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellPhenotypeCategory(name)")
    session.run(query)
    get_count_of_type("CellPhenotypeCategory", session)

Added 7 nodes of type CellPhenotypeCategory


In [12]:
# Adding CellularRegion from cellular_regions.csv

csv_file_path = "file:///%s" % get_csv_path("cellular_regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellularRegion { id:row.ID, name: row.Cellular_region, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellularRegion) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellularRegion(name)")
    session.run(query)
    get_count_of_type("CellularRegion", session)

Added 10 nodes of type CellularRegion


In [13]:
# Adding NeuralStructure from objects_of_intrest.csv

csv_file_path = "file:///%s" % get_csv_path("objects_of_interest.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:NeuralStructure { id:row.ID, name: row.Object_of_interest, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:NeuralStructure) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :NeuralStructure(name)")
    session.run(query)
    get_count_of_type("NeuralStructure", session)

Added 14 nodes of type NeuralStructure


In [15]:
# Relationship PHENOTYPE_CATEGORY between CellPhenotype and CellPhenotypeCategory

csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellPhenotype { id: row.ID})
        MATCH (c:CellPhenotypeCategory { id: row.Phenotype_category })
        MERGE (a)-[:PHENOTYPE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE_CATEGORY", session)

Added 210 relationships of type PHENOTYPE_CATEGORY


In [16]:
# Relationship PHENOTYPE between Cell and CellPhenotype

csv_file_path = "file:///%s" % get_csv_path("celltype_phenotype.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Cell { id: row.CellID})
        MATCH (c:CellPhenotype { id: row.PhenotypeID })
        MERGE (a)-[:PHENOTYPE]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE", session)

Added 322 relationships of type PHENOTYPE
